In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import contextily as ctx
import pyproj
from shapely.geometry import Point, LineString
from zipfile import ZipFile, Path
import datetime
from matplotlib.colors import TwoSlopeNorm

In [2]:
with ZipFile("C:\\Users\\ASUS\\ZTG\\GTFS.zip") as myzip:
    stops_df = pd.read_csv(myzip.open("stops.txt"), dtype={ 
    'stop_id': 'str', 
    'stop_code': 'str',
    'stop_name': 'str',
    'stop_desc' : 'str',                                              
    'stop_lat': 'float',
    'stop_lon': 'float',
    'location_type': 'Int64',
    'parent_station': 'str',
    'wheelchair_boarding': 'str', 
    'platform_code': 'str',
    'zone_id': 'str',
    'level_id' : 'str'
    })
    
    
    stop_times_df = pd.read_csv(myzip.open("stop_times.txt"), dtype={
        'trip_id': 'str',
        'arrival_time': 'str',
        'stop_id': 'str', 
        'departure_time': 'str', 
        'stop_id': 'str',
        'stop_sequence': 'Int64',
        'stop_headsign': 'str',
        'pickup_type': 'Int64',
        'drop_off_type': 'Int64',
    })
    
    
    
    routes_df = pd.read_csv(myzip.open("routes.txt"), dtype={
        'route_id': 'str',  
        'agency_id': 'str',  
        'route_short_name': 'str',  
        'route_long_name': 'str', 
        'route_desc': 'str', 
        'route_type': 'Int64',
        'route_color': 'str',  
        'route_text_color': 'str', 
        'rout_desc': 'str'
    })
    
    trips_df = pd.read_csv(myzip.open("trips.txt"), dtype={
        'route_id': 'str', 
        'service_id': 'str',  
        'trip_id': 'str',
        'shape_id': 'str', 
        'trip_headsign': 'str',
        'trip_short_name': 'str',
        'direction_id': 'Int64',  
        'block_id': 'str',
        'shape_id': 'str',
        'wheelchair_accessible': 'str',  
        'bikes_allowed': 'str'
    })
    
    shapes_df = pd.read_csv(myzip.open("shapes.txt"), dtype={
        'shape_id': 'str', 
        'shape_pt_lat': 'float', 
        'shape_pt_lon': 'float',  
        'shape_pt_sequence': 'Int64'
    })
    
    calendar_df = pd.read_csv(myzip.open("calendar.txt"), dtype={
        'service_id': 'str',  
        'monday': 'bool',  
        'tuesday': 'bool',  
        'wednesday': 'bool',  
        'thursday': 'bool',  
        'friday': 'bool', 
        'saturday': 'bool',  
        'sunday': 'bool',  
        'start_date': 'str', 
        'end_date': 'str',
    })
    
    calendar_dates_df = pd.read_csv(myzip.open("calendar_dates.txt"), dtype={
        'service_id': 'str',  
        'date': 'str',
        'exception_type': 'Int64',
    })
    
    agency_df = pd.read_csv(myzip.open("agency.txt"), dtype={
        'agency_id': 'str', 
        'agency_name': 'str', 
        'agency_url': 'str',  
        'agency_timezone': 'str',
        'agency_lang': 'str', 
        'agency_phone': 'str',
    })

In [3]:
show_date_str = "2023-11-27"

date = datetime.datetime.strptime(show_date_str, "%Y-%m-%d")
date_string = date.strftime("%Y%m%d")
day_of_week_name = date.strftime('%A').lower()

services_for_day_1 = calendar_df[(calendar_df[day_of_week_name]) & (date_string >= calendar_df.start_date) & (date_string <= calendar_df.end_date)].service_id.to_numpy()

services_added_for_day = calendar_dates_df[(calendar_dates_df.date == date_string) & (calendar_dates_df.exception_type == 1)].service_id.to_numpy()
services_removed_for_day = calendar_dates_df[(calendar_dates_df.date == date_string) & (calendar_dates_df.exception_type == 2)].service_id.to_numpy()
services_for_day_2 = np.concatenate([services_for_day_1, services_added_for_day])
services_for_day = np.setdiff1d(services_for_day_2, services_removed_for_day)

trips_for_day = trips_df[trips_df.service_id.isin(services_for_day)]
berlin_bus_route_ids = routes_df[(routes_df['route_type'] == 700) | (routes_df['route_type'] == 3) ].route_id.unique()
day_trip_buses = trips_for_day[trips_for_day.route_id.isin(berlin_bus_route_ids)]

In [4]:
day_trip_buses

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,22739_700,1,222549500,"Kremmen, Bahnhof",1,0,NaN,672,0,0
1,22739_700,2,222549494,"Linum, Küsterstege",11,0,NaN,673,0,0
2,22739_700,2,222549493,"Linum, Küsterstege",13,0,NaN,673,0,0
3,22739_700,1,222549498,"Kremmen, Bahnhof",15,0,NaN,671,0,0
4,22739_700,2,222549492,"Linum, Küsterstege",17,0,NaN,673,0,0
...,...,...,...,...,...,...,...,...,...,...
249567,17047_700,102,223030344,"Hagenwerder, Bahnhof",4140,1,NaN,14008,0,0
249589,17040_700,102,205605916,"Cottbus, Hauptbahnhof",879,0,NaN,13999,0,0
249591,17040_700,102,205605917,"Forst, Bahnhof",878,1,NaN,14000,0,0
249602,17039_700,233,222747498,S Potsdam Hauptbahnhof/Nord ILB,6157,0,NaN,13996,0,0


In [5]:
stop_data_shape = pd.merge(day_trip_buses, stop_times_df[['trip_id','stop_id','stop_sequence']], on='trip_id')

In [6]:
stops_gdf = gpd.GeoDataFrame(stops_df, geometry = gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)).set_crs(epsg=4326)

In [23]:
shapes = shapes_df[["shape_id", "shape_pt_lat", "shape_pt_lon"]].groupby("shape_id").agg(list).apply(lambda x: LineString(zip(x[1], x[0])), axis=1)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12224\2667917217.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  shapes = shapes_df[["shape_id", "shape_pt_lat", "shape_pt_lon"]].groupby("shape_id").agg(list).apply(lambda x: LineString(zip(x[1], x[0])), axis=1)


In [31]:
shapes = gpd.GeoDataFrame( data=shapes.index, geometry = shapes.values, crs=4326)

TypeError: Input must be valid geometry objects: ['1'
 <LINESTRING (12.4 53, 12.4 53, 12.4 53, 12.4 52.9, 12.4 52.9, 12.4 52.9, 12....>]

In [28]:
shapes['shape_id'] = shapes.shape_id.astype(str)

In [32]:
shapes

,shape_id,geometry
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950..."
1,10,"LINESTRING (12.38104 52.95109, 12.38163 52.950..."
2,100,"LINESTRING (12.45170 52.85273, 12.46104 52.868..."
3,1000,"LINESTRING (13.61123 52.51212, 13.61122 52.512..."
4,10000,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."
...,...,...
14198,9995,"LINESTRING (14.06245 52.28595, 14.06271 52.285..."
14199,9996,"LINESTRING (14.24359 52.17309, 14.24314 52.170..."
14200,9997,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."
14201,9998,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."


In [33]:
shapes = shapes.rename(columns={'geometry': 'geometry_shapes'})
stops_gdf = stops_gdf.rename(columns={'geometry': 'geometry_stops'})

In [8]:
df1 = pd.merge(stop_data_shape, stops_gdf[['stop_id','stop_name','geometry_stops']], on='stop_id')

In [9]:
df2 = pd.merge(df1, routes_df[['route_id','route_short_name']], on='route_id')

In [10]:
df2

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,stop_id,stop_sequence,stop_name,geometry_stops,route_short_name
0,22739_700,1,222549500,"Kremmen, Bahnhof",1,0,NaN,672,0,0,de:12068:900205112::1,0,"Fehrbellin, Postplatz",POINT (12.77192 52.81150),758
1,22739_700,2,222549494,"Linum, Küsterstege",11,0,NaN,673,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150),758
2,22739_700,2,222549493,"Linum, Küsterstege",13,0,NaN,673,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150),758
3,22739_700,1,222549498,"Kremmen, Bahnhof",15,0,NaN,671,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150),758
4,22739_700,2,222549492,"Linum, Küsterstege",17,0,NaN,673,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150),758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142797,17039_700,233,222747513,"Beelitz, Stadt Bhf",6156,1,NaN,13995,0,0,de:12069:900220303:1:51,2,"Ferch-Lienewitz, Bahnhof",POINT (12.96417 52.30373),RB33
1142798,17039_700,233,222747498,S Potsdam Hauptbahnhof/Nord ILB,6157,0,NaN,13996,0,0,de:12069:900220573:1:50,2,"Caputh, Schwielowsee Bhf",POINT (12.98041 52.34101),RB33
1142799,17039_700,233,222747513,"Beelitz, Stadt Bhf",6156,1,NaN,13995,0,0,de:12069:900220573:1:50,1,"Caputh, Schwielowsee Bhf",POINT (12.98041 52.34101),RB33
1142800,17039_700,233,222747498,S Potsdam Hauptbahnhof/Nord ILB,6157,0,NaN,13996,0,0,de:12054:900230005::3,3,S Potsdam Hauptbahnhof/Nord ILB,POINT (13.06794 52.39266),RB33


In [11]:
req_columns = ["shape_id", "stop_sequence", "stop_id", "geometry_stops"]
add_columns = ["route_id", "route_short_name","direction_id", "stop_name"]

In [13]:
df_shape_stop = df2[req_columns + add_columns].drop_duplicates()

In [14]:
df_shape_stop

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name
0,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz"
1,673,1,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz"
3,671,1,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz"
8,674,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz"
9,682,12,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,1,"Fehrbellin, Postplatz"
...,...,...,...,...,...,...,...,...
1142797,13995,2,de:12069:900220303:1:51,POINT (12.96417 52.30373),17039_700,RB33,1,"Ferch-Lienewitz, Bahnhof"
1142798,13996,2,de:12069:900220573:1:50,POINT (12.98041 52.34101),17039_700,RB33,0,"Caputh, Schwielowsee Bhf"
1142799,13995,1,de:12069:900220573:1:50,POINT (12.98041 52.34101),17039_700,RB33,1,"Caputh, Schwielowsee Bhf"
1142800,13996,3,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,0,S Potsdam Hauptbahnhof/Nord ILB


In [15]:
df3 = pd.merge(df_shape_stop, shapes_gdf[['shape_id','geometry_shapes']], on='shape_id')

In [16]:
df3

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes
0,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz",POINT (12.77180 52.81151)
1,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz",POINT (12.77201 52.81160)
2,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz",POINT (12.77202 52.81159)
3,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz",POINT (12.77282 52.81108)
4,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz",POINT (12.77319 52.81088)
...,...,...,...,...,...,...,...,...,...
69174336,13995,0,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,1,S Potsdam Hauptbahnhof/Nord ILB,POINT (12.96760 52.24332)
69174337,13995,0,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,1,S Potsdam Hauptbahnhof/Nord ILB,POINT (12.96723 52.24164)
69174338,13995,0,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,1,S Potsdam Hauptbahnhof/Nord ILB,POINT (12.96709 52.24107)
69174339,13995,0,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,1,S Potsdam Hauptbahnhof/Nord ILB,POINT (12.96701 52.24076)


In [34]:
df4 = pd.merge(df_shape_stop, shapes[['shape_id','geometry_shapes']], on='shape_id')

In [35]:
df4

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes
0,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz","LINESTRING (12.77180 52.81151, 12.77201 52.811..."
1,672,1,de:12068:900205155::1,POINT (12.77756 52.80911),22739_700,758,0,"Fehrbellin, Berliner Allee","LINESTRING (12.77180 52.81151, 12.77201 52.811..."
2,672,2,de:12068:900205110::1,POINT (12.78266 52.80717),22739_700,758,0,"Fehrbellin, Siedlung","LINESTRING (12.77180 52.81151, 12.77201 52.811..."
3,672,3,de:12068:900205576::1,POINT (12.79852 52.80087),22739_700,758,0,"Tarmow, Ausbau","LINESTRING (12.77180 52.81151, 12.77201 52.811..."
4,672,4,de:12068:900205577::1,POINT (12.80582 52.79483),22739_700,758,0,"Tarmow, Gaststätte","LINESTRING (12.77180 52.81151, 12.77201 52.811..."
...,...,...,...,...,...,...,...,...,...
165795,13996,3,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,0,S Potsdam Hauptbahnhof/Nord ILB,"LINESTRING (12.96672 52.23936, 12.96701 52.240..."
165796,13995,3,de:12069:900220938:1:50,POINT (12.96723 52.24156),17039_700,RB33,1,"Beelitz, Stadt Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341..."
165797,13995,2,de:12069:900220303:1:51,POINT (12.96417 52.30373),17039_700,RB33,1,"Ferch-Lienewitz, Bahnhof","LINESTRING (13.06794 52.39266, 12.98041 52.341..."
165798,13995,1,de:12069:900220573:1:50,POINT (12.98041 52.34101),17039_700,RB33,1,"Caputh, Schwielowsee Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341..."


In [36]:
df4["cut_distance_stop_point"] = df4[["geometry_stops", "geometry_shapes"]].apply(lambda x: x[1].project(x[0], normalized=True), axis=1)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12224\3976018568.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df4["cut_distance_stop_point"] = df4[["geometry_stops", "geometry_shapes"]].apply(lambda x: x[1].project(x[0], normalized=True), axis=1)
C:\Users\ASUS\miniconda3\envs\geo_ds\lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point_normalized
  return lib.line_locate_point_normalized(line, other)


In [37]:
df4

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,cut_distance_stop_point
0,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.000384
1,672,1,de:12068:900205155::1,POINT (12.77756 52.80911),22739_700,758,0,"Fehrbellin, Berliner Allee","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.021983
2,672,2,de:12068:900205110::1,POINT (12.78266 52.80717),22739_700,758,0,"Fehrbellin, Siedlung","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.040920
3,672,3,de:12068:900205576::1,POINT (12.79852 52.80087),22739_700,758,0,"Tarmow, Ausbau","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.100817
4,672,4,de:12068:900205577::1,POINT (12.80582 52.79483),22739_700,758,0,"Tarmow, Gaststätte","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.134106
...,...,...,...,...,...,...,...,...,...,...
165795,13996,3,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,0,S Potsdam Hauptbahnhof/Nord ILB,"LINESTRING (12.96672 52.23936, 12.96701 52.240...",1.000000
165796,13995,3,de:12069:900220938:1:50,POINT (12.96723 52.24156),17039_700,RB33,1,"Beelitz, Stadt Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.989590
165797,13995,2,de:12069:900220303:1:51,POINT (12.96417 52.30373),17039_700,RB33,1,"Ferch-Lienewitz, Bahnhof","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.672032
165798,13995,1,de:12069:900220573:1:50,POINT (12.98041 52.34101),17039_700,RB33,1,"Caputh, Schwielowsee Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.469645


In [39]:
df4["projected_stop_point"] = df4[["geometry_shapes", "cut_distance_stop_point"]].apply(lambda x: x[0].interpolate(x[1], normalized=True), axis=1)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12224\3103908401.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df4["projected_stop_point"] = df4[["geometry_shapes", "cut_distance_stop_point"]].apply(lambda x: x[0].interpolate(x[1], normalized=True), axis=1)


In [40]:
df4

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,cut_distance_stop_point,projected_stop_point
0,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.000384,POINT (12.77190088470588 52.81155789882353)
1,672,1,de:12068:900205155::1,POINT (12.77756 52.80911),22739_700,758,0,"Fehrbellin, Berliner Allee","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.021983,POINT (12.77757363120968 52.80914421019705)
2,672,2,de:12068:900205110::1,POINT (12.78266 52.80717),22739_700,758,0,"Fehrbellin, Siedlung","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.040920,POINT (12.78267583325751 52.8072073637092)
3,672,3,de:12068:900205576::1,POINT (12.79852 52.80087),22739_700,758,0,"Tarmow, Ausbau","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.100817,POINT (12.79852613264651 52.80088217738065)
4,672,4,de:12068:900205577::1,POINT (12.80582 52.79483),22739_700,758,0,"Tarmow, Gaststätte","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.134106,POINT (12.80586947312944 52.79489687776231)
...,...,...,...,...,...,...,...,...,...,...,...
165795,13996,3,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,0,S Potsdam Hauptbahnhof/Nord ILB,"LINESTRING (12.96672 52.23936, 12.96701 52.240...",1.000000,POINT (13.06794 52.39266)
165796,13995,3,de:12069:900220938:1:50,POINT (12.96723 52.24156),17039_700,RB33,1,"Beelitz, Stadt Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.989590,POINT (12.96721021646536 52.2415621774246)
165797,13995,2,de:12069:900220303:1:51,POINT (12.96417 52.30373),17039_700,RB33,1,"Ferch-Lienewitz, Bahnhof","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.672032,POINT (12.96415977389554 52.30373693761258)
165798,13995,1,de:12069:900220573:1:50,POINT (12.98041 52.34101),17039_700,RB33,1,"Caputh, Schwielowsee Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.469645,POINT (12.980411 52.341005)


In [41]:
df_shape = shapes[shapes.shape_id.isin(df2.shape_id.unique())]

In [43]:
df_shape

,shape_id,geometry_shapes
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950..."
1,10,"LINESTRING (12.38104 52.95109, 12.38163 52.950..."
4,10000,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."
5,10001,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."
6,10002,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."
...,...,...
14198,9995,"LINESTRING (14.06245 52.28595, 14.06271 52.285..."
14199,9996,"LINESTRING (14.24359 52.17309, 14.24314 52.170..."
14200,9997,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."
14201,9998,"LINESTRING (14.05871 52.36673, 14.05862 52.366..."


In [45]:
from shapely.geometry import LineString, MultiPoint

In [46]:
df_shape["list_of_points"] = df_shape.geometry_shapes.apply(lambda x: list(MultiPoint(x.coords).geoms))

C:\Users\ASUS\miniconda3\envs\geo_ds\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [47]:
df_shape

,shape_id,geometry_shapes,list_of_points
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950...","[POINT (12.381042 52.951089), POINT (12.381628..."
1,10,"LINESTRING (12.38104 52.95109, 12.38163 52.950...","[POINT (12.381042 52.951089), POINT (12.381628..."
4,10000,"LINESTRING (14.05871 52.36673, 14.05862 52.366...","[POINT (14.058707 52.366733), POINT (14.058621..."
5,10001,"LINESTRING (14.05871 52.36673, 14.05862 52.366...","[POINT (14.058707 52.366733), POINT (14.058621..."
6,10002,"LINESTRING (14.05871 52.36673, 14.05862 52.366...","[POINT (14.058707 52.366733), POINT (14.058621..."
...,...,...,...
14198,9995,"LINESTRING (14.06245 52.28595, 14.06271 52.285...","[POINT (14.062449 52.285947), POINT (14.06271 ..."
14199,9996,"LINESTRING (14.24359 52.17309, 14.24314 52.170...","[POINT (14.243589 52.17309), POINT (14.24314 5..."
14200,9997,"LINESTRING (14.05871 52.36673, 14.05862 52.366...","[POINT (14.058707 52.366733), POINT (14.058621..."
14201,9998,"LINESTRING (14.05871 52.36673, 14.05862 52.366...","[POINT (14.058707 52.366733), POINT (14.058621..."


In [48]:
df_shape_exp = df_shape.explode("list_of_points")

In [49]:
df_shape_exp

,shape_id,geometry_shapes,list_of_points
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950...",POINT (12.381042 52.951089)
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950...",POINT (12.381628 52.95093)
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950...",POINT (12.381207 52.950344)
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950...",POINT (12.380649 52.949537)
0,1,"LINESTRING (12.38104 52.95109, 12.38163 52.950...",POINT (12.381094 52.949392)
...,...,...,...
14202,9999,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",POINT (14.06418 52.292739)
14202,9999,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",POINT (14.064448 52.292772)
14202,9999,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",POINT (14.064586 52.29283)
14202,9999,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",POINT (14.064616 52.292963)


In [50]:
df_shape_exp["projected_line_points"] = df_shape_exp[["geometry_shapes", "list_of_points"]].apply(lambda x: x[0].project(x[1], normalized=True), axis=1)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12224\2032372846.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_shape_exp["projected_line_points"] = df_shape_exp[["geometry_shapes", "list_of_points"]].apply(lambda x: x[0].project(x[1], normalized=True), axis=1)
C:\Users\ASUS\miniconda3\envs\geo_ds\lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point_normalized
  return lib.line_locate_point_normalized(line, other)


In [51]:
df4.rename({ "projected_stop_point": "geometry", "cut_distance_stop_point": "normalized_distance_along_shape"},axis=1,inplace=True)
df4["cut_flag"] = True

In [52]:
df4

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,normalized_distance_along_shape,geometry,cut_flag
0,672,0,de:12068:900205112::1,POINT (12.77192 52.81150),22739_700,758,0,"Fehrbellin, Postplatz","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.000384,POINT (12.77190088470588 52.81155789882353),True
1,672,1,de:12068:900205155::1,POINT (12.77756 52.80911),22739_700,758,0,"Fehrbellin, Berliner Allee","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.021983,POINT (12.77757363120968 52.80914421019705),True
2,672,2,de:12068:900205110::1,POINT (12.78266 52.80717),22739_700,758,0,"Fehrbellin, Siedlung","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.040920,POINT (12.78267583325751 52.8072073637092),True
3,672,3,de:12068:900205576::1,POINT (12.79852 52.80087),22739_700,758,0,"Tarmow, Ausbau","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.100817,POINT (12.79852613264651 52.80088217738065),True
4,672,4,de:12068:900205577::1,POINT (12.80582 52.79483),22739_700,758,0,"Tarmow, Gaststätte","LINESTRING (12.77180 52.81151, 12.77201 52.811...",0.134106,POINT (12.80586947312944 52.79489687776231),True
...,...,...,...,...,...,...,...,...,...,...,...,...
165795,13996,3,de:12054:900230005::3,POINT (13.06794 52.39266),17039_700,RB33,0,S Potsdam Hauptbahnhof/Nord ILB,"LINESTRING (12.96672 52.23936, 12.96701 52.240...",1.000000,POINT (13.06794 52.39266),True
165796,13995,3,de:12069:900220938:1:50,POINT (12.96723 52.24156),17039_700,RB33,1,"Beelitz, Stadt Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.989590,POINT (12.96721021646536 52.2415621774246),True
165797,13995,2,de:12069:900220303:1:51,POINT (12.96417 52.30373),17039_700,RB33,1,"Ferch-Lienewitz, Bahnhof","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.672032,POINT (12.96415977389554 52.30373693761258),True
165798,13995,1,de:12069:900220573:1:50,POINT (12.98041 52.34101),17039_700,RB33,1,"Caputh, Schwielowsee Bhf","LINESTRING (13.06794 52.39266, 12.98041 52.341...",0.469645,POINT (12.980411 52.341005),True


In [53]:
df_shape_exp = df_shape_exp[["shape_id", "list_of_points", "projected_line_points"]]
df_shape_exp.rename({ "list_of_points": "geometry", "projected_line_points": "normalized_distance_along_shape"}, axis=1, inplace=True)
df_shape_exp["cut_flag"] = False

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_12224\1424701205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shape_exp.rename({ "list_of_points": "geometry", "projected_line_points": "normalized_distance_along_shape"}, axis=1, inplace=True)


In [54]:
df_shape_exp

,shape_id,geometry,normalized_distance_along_shape,cut_flag
0,1,POINT (12.381042 52.951089),0.000000,False
0,1,POINT (12.381628 52.95093),0.004403,False
0,1,POINT (12.381207 52.950344),0.009636,False
0,1,POINT (12.380649 52.949537),0.016751,False
0,1,POINT (12.381094 52.949392),0.020145,False
...,...,...,...,...
14202,9999,POINT (14.06418 52.292739),0.996401,False
14202,9999,POINT (14.064448 52.292772),0.997967,False
14202,9999,POINT (14.064586 52.29283),0.998836,False
14202,9999,POINT (14.064616 52.292963),0.999627,False


In [55]:
gdf = pd.concat([df4, df_shape_exp], ignore_index=False)
gdf.sort_values(["shape_id", "normalized_distance_along_shape"], inplace=True)
gdf.reset_index(inplace=True, drop=True)

C:\Users\ASUS\miniconda3\envs\geo_ds\lib\site-packages\geopandas\array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(


In [56]:
gdf

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,normalized_distance_along_shape,geometry,cut_flag
0,1,0,de:12068:900205122::2,POINT (12.38099 52.95107),22684_700,701,0,"Kyritz, Lindenschule","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.000000,POINT (12.381042 52.951089),True
1,1,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.000000,POINT (12.381042 52.951089),False
2,1,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.004403,POINT (12.381628 52.95093),False
3,1,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.009636,POINT (12.381207 52.950344),False
4,1,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.016751,POINT (12.380649 52.949537),False
...,...,...,...,...,...,...,...,...,...,...,...,...
3306960,9999,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.997967,POINT (14.064448 52.292772),False
3306961,9999,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.998836,POINT (14.064586 52.29283),False
3306962,9999,<NA>,NaN,None,NaN,NaN,<NA>,NaN,None,0.999627,POINT (14.064616 52.292963),False
3306963,9999,16,de:12067:900310040::1,POINT (14.06464 52.29303),23869_700,431,0,"Bad Saarow, Bahnhof","LINESTRING (14.05871 52.36673, 14.05862 52.366...",1.000000,POINT (14.064609 52.293027),True


In [57]:
cuts = gdf.where(gdf.cut_flag).dropna(subset="cut_flag")
cuts = cuts.astype({"shape_id": str, "stop_sequence": int, "direction_id": int})
cuts[["end_stop_id", "end_stop_name"]] = cuts.groupby("shape_id")[['stop_id', "stop_name"]].shift(-1)

In [58]:
cuts

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,normalized_distance_along_shape,geometry,cut_flag,end_stop_id,end_stop_name
0,1,0,de:12068:900205122::2,POINT (12.38099 52.95107),22684_700,701,0,"Kyritz, Lindenschule","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.000000,POINT (12.381042 52.951089),True,de:12068:900205147::2,"Kyritz, Werner Str."
10,1,1,de:12068:900205147::2,POINT (12.38370 52.94852),22684_700,701,0,"Kyritz, Werner Str.","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.040084,POINT (12.38370697796449 52.94854255566217),True,de:12068:900205042::1,"Kyritz, Straße der Jugend"
20,1,2,de:12068:900205042::1,POINT (12.38394 52.94588),22684_700,701,0,"Kyritz, Straße der Jugend","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.066848,POINT (12.3839907881345 52.94589460792862),True,de:12068:900205388::1,"Kyritz, Prignitz-Center"
36,1,3,de:12068:900205388::1,POINT (12.38537 52.94183),22684_700,701,0,"Kyritz, Prignitz-Center","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.115988,POINT (12.3854407474088 52.94185970147014),True,770000205031,"Kyritz, Hagenstr."
56,1,4,770000205031,POINT (12.39265 52.93872),22684_700,701,0,"Kyritz, Hagenstr.","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.175484,POINT (12.39265336596214 52.93872433602019),True,de:12068:900205139::1,"Kyritz, Bahnhof"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306841,9999,12,de:12067:900310703::1,POINT (14.12457 52.30430),23869_700,431,0,Alt Golm (LOS),"LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.603107,POINT (14.124551 52.304321),True,de:12067:900310979::1,"Neu Golm, Ausbau"
3306876,9999,13,de:12067:900310979::1,POINT (14.09969 52.30313),23869_700,431,0,"Neu Golm, Ausbau","LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.760591,POINT (14.09969086594572 52.30312161620078),True,de:12067:900310946::1,Neu Golm
3306906,9999,14,de:12067:900310946::1,POINT (14.09032 52.30352),23869_700,431,0,Neu Golm,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.815626,POINT (14.09031374056545 52.30350432418924),True,de:12067:900310110::1,"Bad Saarow, Jägerstr."
3306931,9999,15,de:12067:900310110::1,POINT (14.07104 52.29558),23869_700,431,0,"Bad Saarow, Jägerstr.","LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.938266,POINT (14.07106396946109 52.29551472479906),True,de:12067:900310040::1,"Bad Saarow, Bahnhof"


In [59]:
segment_geometries = []
for shape_id in cuts.shape_id.drop_duplicates():
    cut_idx = cuts[cuts.shape_id == shape_id].index
    for i, cut in enumerate(cut_idx[:-1]):
        segment_geometries.append(LineString(gdf.iloc[cut_idx[i]:cut_idx[i+1]+1].geometry))

In [60]:
segment_df = cuts.dropna(subset="end_stop_id", axis=0)

In [61]:
segment_df

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,normalized_distance_along_shape,geometry,cut_flag,end_stop_id,end_stop_name
0,1,0,de:12068:900205122::2,POINT (12.38099 52.95107),22684_700,701,0,"Kyritz, Lindenschule","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.000000,POINT (12.381042 52.951089),True,de:12068:900205147::2,"Kyritz, Werner Str."
10,1,1,de:12068:900205147::2,POINT (12.38370 52.94852),22684_700,701,0,"Kyritz, Werner Str.","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.040084,POINT (12.38370697796449 52.94854255566217),True,de:12068:900205042::1,"Kyritz, Straße der Jugend"
20,1,2,de:12068:900205042::1,POINT (12.38394 52.94588),22684_700,701,0,"Kyritz, Straße der Jugend","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.066848,POINT (12.3839907881345 52.94589460792862),True,de:12068:900205388::1,"Kyritz, Prignitz-Center"
36,1,3,de:12068:900205388::1,POINT (12.38537 52.94183),22684_700,701,0,"Kyritz, Prignitz-Center","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.115988,POINT (12.3854407474088 52.94185970147014),True,770000205031,"Kyritz, Hagenstr."
56,1,4,770000205031,POINT (12.39265 52.93872),22684_700,701,0,"Kyritz, Hagenstr.","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.175484,POINT (12.39265336596214 52.93872433602019),True,de:12068:900205139::1,"Kyritz, Bahnhof"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306802,9999,11,de:12067:900310915::2,POINT (14.11785 52.31673),23869_700,431,0,"Alt Golm, Deponie","LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.498795,POINT (14.11779023917474 52.3166742319787),True,de:12067:900310703::1,Alt Golm (LOS)
3306841,9999,12,de:12067:900310703::1,POINT (14.12457 52.30430),23869_700,431,0,Alt Golm (LOS),"LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.603107,POINT (14.124551 52.304321),True,de:12067:900310979::1,"Neu Golm, Ausbau"
3306876,9999,13,de:12067:900310979::1,POINT (14.09969 52.30313),23869_700,431,0,"Neu Golm, Ausbau","LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.760591,POINT (14.09969086594572 52.30312161620078),True,de:12067:900310946::1,Neu Golm
3306906,9999,14,de:12067:900310946::1,POINT (14.09032 52.30352),23869_700,431,0,Neu Golm,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.815626,POINT (14.09031374056545 52.30350432418924),True,de:12067:900310110::1,"Bad Saarow, Jägerstr."


In [62]:
segment_gdf = gpd.GeoDataFrame(segment_df, geometry=segment_geometries)

In [63]:
segment_gdf

,shape_id,stop_sequence,stop_id,geometry_stops,route_id,route_short_name,direction_id,stop_name,geometry_shapes,normalized_distance_along_shape,geometry,cut_flag,end_stop_id,end_stop_name
0,1,0,de:12068:900205122::2,POINT (12.38099 52.95107),22684_700,701,0,"Kyritz, Lindenschule","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.000000,"LINESTRING (12.38104 52.95109, 12.38104 52.951...",True,de:12068:900205147::2,"Kyritz, Werner Str."
10,1,1,de:12068:900205147::2,POINT (12.38370 52.94852),22684_700,701,0,"Kyritz, Werner Str.","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.040084,"LINESTRING (12.38371 52.94854, 12.38372 52.948...",True,de:12068:900205042::1,"Kyritz, Straße der Jugend"
20,1,2,de:12068:900205042::1,POINT (12.38394 52.94588),22684_700,701,0,"Kyritz, Straße der Jugend","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.066848,"LINESTRING (12.38399 52.94589, 12.38402 52.945...",True,de:12068:900205388::1,"Kyritz, Prignitz-Center"
36,1,3,de:12068:900205388::1,POINT (12.38537 52.94183),22684_700,701,0,"Kyritz, Prignitz-Center","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.115988,"LINESTRING (12.38544 52.94186, 12.38556 52.941...",True,770000205031,"Kyritz, Hagenstr."
56,1,4,770000205031,POINT (12.39265 52.93872),22684_700,701,0,"Kyritz, Hagenstr.","LINESTRING (12.38104 52.95109, 12.38163 52.950...",0.175484,"LINESTRING (12.39265 52.93872, 12.39302 52.938...",True,de:12068:900205139::1,"Kyritz, Bahnhof"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3306802,9999,11,de:12067:900310915::2,POINT (14.11785 52.31673),23869_700,431,0,"Alt Golm, Deponie","LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.498795,"LINESTRING (14.11779 52.31667, 14.11792 52.316...",True,de:12067:900310703::1,Alt Golm (LOS)
3306841,9999,12,de:12067:900310703::1,POINT (14.12457 52.30430),23869_700,431,0,Alt Golm (LOS),"LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.603107,"LINESTRING (14.12455 52.30432, 14.12455 52.304...",True,de:12067:900310979::1,"Neu Golm, Ausbau"
3306876,9999,13,de:12067:900310979::1,POINT (14.09969 52.30313),23869_700,431,0,"Neu Golm, Ausbau","LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.760591,"LINESTRING (14.09969 52.30312, 14.09910 52.303...",True,de:12067:900310946::1,Neu Golm
3306906,9999,14,de:12067:900310946::1,POINT (14.09032 52.30352),23869_700,431,0,Neu Golm,"LINESTRING (14.05871 52.36673, 14.05862 52.366...",0.815626,"LINESTRING (14.09031 52.30350, 14.09001 52.303...",True,de:12067:900310110::1,"Bad Saarow, Jägerstr."


In [65]:
segment_gdf.drop(["geometry_shapes", "cut_flag", "normalized_distance_along_shape", "geometry_stops"], axis=1, inplace=True)
segment_gdf.crs = "EPSG:4326"

In [66]:
def code(gdf):
    gdf.index=list(range(0,len(gdf)))
    gdf.crs = {'init':'epsg:4326'}
    lat_referece = gdf.geometry[0].coords[0][1]
    lon_reference = gdf.geometry[0].coords[0][0]

    zone = utm.from_latlon(lat_referece, lon_reference)
    #The EPSG code is 32600+zone for positive latitudes and 32700+zone for negatives.
    if lat_referece <0:
        epsg_code = 32700 + zone[2]
    else:
        epsg_code = 32600 + zone[2]
        
    return epsg_code

In [69]:
!pip install utm

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6103 sha256=b6d1001ff8fa92470fd7d5226564e721bd4f2c3854cc31c894faeb4826ba7c50
  Stored in directory: c:\users\asus\appdata\local\pip\cache\wheels\75\f0\dd\39dfc9436520412399934d4a8566fe5c0cb8cc2568a1196057
Successfully built utm


In [70]:
import utm
segment_gdf['distance_m'] = segment_gdf.to_crs(code(segment_gdf)).length

ProjError: x, y, z, and time must be same size

In [71]:
segment_gdf

,shape_id,stop_sequence,stop_id,route_id,route_short_name,direction_id,stop_name,geometry,end_stop_id,end_stop_name
0,1,0,de:12068:900205122::2,22684_700,701,0,"Kyritz, Lindenschule","LINESTRING (12.38104 52.95109, 12.38104 52.951...",de:12068:900205147::2,"Kyritz, Werner Str."
1,1,1,de:12068:900205147::2,22684_700,701,0,"Kyritz, Werner Str.","LINESTRING (12.38371 52.94854, 12.38372 52.948...",de:12068:900205042::1,"Kyritz, Straße der Jugend"
2,1,2,de:12068:900205042::1,22684_700,701,0,"Kyritz, Straße der Jugend","LINESTRING (12.38399 52.94589, 12.38402 52.945...",de:12068:900205388::1,"Kyritz, Prignitz-Center"
3,1,3,de:12068:900205388::1,22684_700,701,0,"Kyritz, Prignitz-Center","LINESTRING (12.38544 52.94186, 12.38556 52.941...",770000205031,"Kyritz, Hagenstr."
4,1,4,770000205031,22684_700,701,0,"Kyritz, Hagenstr.","LINESTRING (12.39265 52.93872, 12.39302 52.938...",de:12068:900205139::1,"Kyritz, Bahnhof"
...,...,...,...,...,...,...,...,...,...,...
156587,9999,11,de:12067:900310915::2,23869_700,431,0,"Alt Golm, Deponie","LINESTRING (14.11779 52.31667, 14.11792 52.316...",de:12067:900310703::1,Alt Golm (LOS)
156588,9999,12,de:12067:900310703::1,23869_700,431,0,Alt Golm (LOS),"LINESTRING (14.12455 52.30432, 14.12455 52.304...",de:12067:900310979::1,"Neu Golm, Ausbau"
156589,9999,13,de:12067:900310979::1,23869_700,431,0,"Neu Golm, Ausbau","LINESTRING (14.09969 52.30312, 14.09910 52.303...",de:12067:900310946::1,Neu Golm
156590,9999,14,de:12067:900310946::1,23869_700,431,0,Neu Golm,"LINESTRING (14.09031 52.30350, 14.09001 52.303...",de:12067:900310110::1,"Bad Saarow, Jägerstr."


In [72]:
segment_gdf['segment_id'] = segment_gdf.stop_id.astype(str) + ' - ' + segment_gdf.end_stop_id.astype(str)
segment_gdf['segment_name'] = segment_gdf.stop_name + ' - ' + segment_gdf.end_stop_name

In [73]:
segment_gdf

,shape_id,stop_sequence,stop_id,route_id,route_short_name,direction_id,stop_name,geometry,end_stop_id,end_stop_name,segment_id,segment_name
0,1,0,de:12068:900205122::2,22684_700,701,0,"Kyritz, Lindenschule","LINESTRING (12.38104 52.95109, 12.38104 52.951...",de:12068:900205147::2,"Kyritz, Werner Str.",de:12068:900205122::2 - de:12068:900205147::2,"Kyritz, Lindenschule - Kyritz, Werner Str."
1,1,1,de:12068:900205147::2,22684_700,701,0,"Kyritz, Werner Str.","LINESTRING (12.38371 52.94854, 12.38372 52.948...",de:12068:900205042::1,"Kyritz, Straße der Jugend",de:12068:900205147::2 - de:12068:900205042::1,"Kyritz, Werner Str. - Kyritz, Straße der Jugend"
2,1,2,de:12068:900205042::1,22684_700,701,0,"Kyritz, Straße der Jugend","LINESTRING (12.38399 52.94589, 12.38402 52.945...",de:12068:900205388::1,"Kyritz, Prignitz-Center",de:12068:900205042::1 - de:12068:900205388::1,"Kyritz, Straße der Jugend - Kyritz, Prignitz-C..."
3,1,3,de:12068:900205388::1,22684_700,701,0,"Kyritz, Prignitz-Center","LINESTRING (12.38544 52.94186, 12.38556 52.941...",770000205031,"Kyritz, Hagenstr.",de:12068:900205388::1 - 770000205031,"Kyritz, Prignitz-Center - Kyritz, Hagenstr."
4,1,4,770000205031,22684_700,701,0,"Kyritz, Hagenstr.","LINESTRING (12.39265 52.93872, 12.39302 52.938...",de:12068:900205139::1,"Kyritz, Bahnhof",770000205031 - de:12068:900205139::1,"Kyritz, Hagenstr. - Kyritz, Bahnhof"
...,...,...,...,...,...,...,...,...,...,...,...,...
156587,9999,11,de:12067:900310915::2,23869_700,431,0,"Alt Golm, Deponie","LINESTRING (14.11779 52.31667, 14.11792 52.316...",de:12067:900310703::1,Alt Golm (LOS),de:12067:900310915::2 - de:12067:900310703::1,"Alt Golm, Deponie - Alt Golm (LOS)"
156588,9999,12,de:12067:900310703::1,23869_700,431,0,Alt Golm (LOS),"LINESTRING (14.12455 52.30432, 14.12455 52.304...",de:12067:900310979::1,"Neu Golm, Ausbau",de:12067:900310703::1 - de:12067:900310979::1,"Alt Golm (LOS) - Neu Golm, Ausbau"
156589,9999,13,de:12067:900310979::1,23869_700,431,0,"Neu Golm, Ausbau","LINESTRING (14.09969 52.30312, 14.09910 52.303...",de:12067:900310946::1,Neu Golm,de:12067:900310979::1 - de:12067:900310946::1,"Neu Golm, Ausbau - Neu Golm"
156590,9999,14,de:12067:900310946::1,23869_700,431,0,Neu Golm,"LINESTRING (14.09031 52.30350, 14.09001 52.303...",de:12067:900310110::1,"Bad Saarow, Jägerstr.",de:12067:900310946::1 - de:12067:900310110::1,"Neu Golm - Bad Saarow, Jägerstr."


In [74]:
col_ordered = ['route_short_name','direction_id','stop_sequence', 'segment_name', 'stop_name', 'end_stop_name', 'segment_id','stop_id', 'end_stop_id','geometry']

In [76]:
segment_gdf = segment_gdf[col_ordered]
segment_gdf.rename( columns=dict(stop_name='start_nm', stop_id='start_id', route_short_name="Bus_Num",direction_id='dir', stop_sequence = 'seq',
                                segment_name = 'seg_nm', end_stop_name = 'end_nm', segment_id = 'seg_id', end_stop_id= 'end_id'),inplace=True)

In [78]:
lines_gdf = gpd.GeoDataFrame(pd.DataFrame(segment_gdf), geometry='geometry')

lines_gdf.to_file("C:\\Users\\ASUS\\ZTG\\lines.shp"", driver='ESRI Shapefile")

In [10]:
shapes_gdf = shapes_gdf.rename(columns={'geometry': 'geometry_shapes'})
stops_gdf = stops_gdf.rename(columns={'geometry': 'geometry_stops'})

In [11]:
df1 = pd.merge(stop_data_shape, stops_gdf[['stop_id','stop_name','geometry_stops']], on='stop_id')

In [13]:
df1

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed,stop_id,stop_sequence,stop_name,geometry_stops
0,22739_700,1,222549500,"Kremmen, Bahnhof",1,0,NaN,672,0,0,de:12068:900205112::1,0,"Fehrbellin, Postplatz",POINT (12.77192 52.81150)
1,22739_700,2,222549494,"Linum, Küsterstege",11,0,NaN,673,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150)
2,22739_700,2,222549493,"Linum, Küsterstege",13,0,NaN,673,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150)
3,22739_700,1,222549498,"Kremmen, Bahnhof",15,0,NaN,671,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150)
4,22739_700,2,222549492,"Linum, Küsterstege",17,0,NaN,673,0,0,de:12068:900205112::1,1,"Fehrbellin, Postplatz",POINT (12.77192 52.81150)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142797,17039_700,233,222747513,"Beelitz, Stadt Bhf",6156,1,NaN,13995,0,0,de:12069:900220303:1:51,2,"Ferch-Lienewitz, Bahnhof",POINT (12.96417 52.30373)
1142798,17039_700,233,222747498,S Potsdam Hauptbahnhof/Nord ILB,6157,0,NaN,13996,0,0,de:12069:900220573:1:50,2,"Caputh, Schwielowsee Bhf",POINT (12.98041 52.34101)
1142799,17039_700,233,222747513,"Beelitz, Stadt Bhf",6156,1,NaN,13995,0,0,de:12069:900220573:1:50,1,"Caputh, Schwielowsee Bhf",POINT (12.98041 52.34101)
1142800,17039_700,233,222747498,S Potsdam Hauptbahnhof/Nord ILB,6157,0,NaN,13996,0,0,de:12054:900230005::3,3,S Potsdam Hauptbahnhof/Nord ILB,POINT (13.06794 52.39266)


In [12]:
df2 = pd.merge(df1, shapes_gdf[['shape_id','geometry_shapes']], on='shape_id')

MemoryError: Unable to allocate 29.3 GiB for an array with shape (10, 393739281) and data type object

MemoryError: Unable to allocate 385. MiB for an array with shape (50400961,) and data type object

In [13]:
coords = shapes_df[["shape_pt_lat", "shape_pt_lon", "shape_pt_sequence"]]
coords_roll_1 = np.roll(coords, 1, axis=0)

segments = pd.DataFrame(np.concatenate([coords_roll_1, coords], axis=1), columns=["start_lat", "start_lng", "start_seq", 
                                                                       "end_lat", "end_lng", "end_seq"])
segments_df = shapes_df[["shape_id"]].join(segments)
segments_df = segments_df[segments_df.end_seq != 0]
segments_df = segments_df.drop(columns=['end_seq']).rename(columns={ "start_seq": "seq" })
segments_df

,shape_id,start_lat,start_lng,seq,end_lat,end_lng
1,28,52.323324,12.568507,0,52.323297,12.568551
2,28,52.323297,12.568551,1,52.323199,12.568418
3,28,52.323199,12.568418,2,52.323038,12.568196
4,28,52.323038,12.568196,3,52.322851,12.567856
5,28,52.322851,12.567856,4,52.322726,12.567634
...,...,...,...,...,...,...
5264762,13634,53.562505,13.256586,206,53.5624,13.257171
5264763,13634,53.5624,13.257171,207,53.562314,13.257607
5264764,13634,53.562314,13.257607,208,53.562216,13.258434
5264765,13634,53.562216,13.258434,209,53.562136,13.259156


In [ ]:
df = df.rename(columns={'A': 'New_Column_Name'})